In [4]:
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import wandb
from sklearn.metrics import accuracy_score

%load_ext autoreload
%autoreload all

from datasets import SimpleFrameSelectionDataset
from utils import read_images, plot_wandb_training_curves
from models import resnet_binary

from vit import ViT_Binary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
images = read_images()

Loading imgs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1664/1664 [03:13<00:00,  8.62it/s]


Saved images to cache.


In [6]:
train_dataset = SimpleFrameSelectionDataset(images, split="train")
test_dataset = SimpleFrameSelectionDataset(images, split="test")

print(f"Train: {len(train_dataset)}")
print(f"Test: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

Train: 1225
Test: 261


In [7]:
run = wandb.init(
    project='corflow-frame_selection', 
    config={
        "architecture": "VitT",
    }
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ViT_Binary() 
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

best_test_loss = np.inf
pbar = tqdm(range(100))
for epoch in pbar:
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = accuracy_score(all_labels, all_preds)

    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_test_loss = running_loss / len(test_loader)
    test_accuracy = accuracy_score(all_labels, all_preds)
    
    pbar.set_description(
        f'Loss: {avg_train_loss:.4f}/{avg_test_loss:.4f} | ' 
        f'Accuracy {train_accuracy:.4f}/{test_accuracy:.4f}'
    )

    wandb.log({
        "test_loss": avg_test_loss, "train_accuracy": train_accuracy,
        "train_loss": avg_train_loss, "test_accuracy": test_accuracy,
        "epoch": epoch
    })

    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        torch.save(model.state_dict(), 'weights/frame_selection.pth')

artifact = wandb.Artifact('model-artifact', type='model')
artifact.add_file('weights/frame_selection.pth')
wandb.log_artifact(artifact)

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: woj-jasinski. Use `wandb login --relogin` to force relogin


Loss: 0.0000/0.9996 | Accuracy 1.0000/0.9310:  95%|█████████████████████████████████████████████████████████████████████████▏   | 95/100 [09:03<00:28,  5.72s/it]


KeyboardInterrupt: 

In [2]:
plot_wandb_training_curves("corflow-frame_selection", metric="accuracy")

NameError: name 'plot_wandb_training_curves' is not defined